In [2]:
import numpy as np
import pandas as pd
import os
import requests
import io
from datetime import datetime, timedelta
import multiprocessing
import arviz as az
import logging
import matplotlib.pyplot as plt
from sklearn.metrics import mean_absolute_error, mean_squared_error
import xgboost as xgb
from scipy.stats import poisson
import sqlite3

# get data
API_KEY = os.getenv("API_KEY")
url = 'https://data-service.beatthebookie.blog/data'
headers = {"x-api-key": API_KEY}

# Function to fetch data for a specific division and season
def fetch_data(division, season):
    params = {
        'division': division,
        'season': season
    }
    response = requests.get(url, headers=headers, params=params)
    if response.status_code == 200:
        return pd.read_json(io.StringIO(response.content.decode('utf-8')))
    else:
        print(f"Error fetching {division} {season}: {response.status_code}")
        print(response.content.decode('utf-8'))
        return pd.DataFrame()

# Fetch data for all combinations
seasons = ['2024_2025', '2023_2024', '2022_2023', '2021_2022', '2020_2021']
divisions = ['Premier League', 'Championship'] #'Bundesliga', 'Serie A', 'La Liga', 'Ligue 1'
dataframes = []

for division in divisions:
    for season in seasons:
        df = fetch_data(division, season)
        if not df.empty:
            dataframes.append(df)

# Combine all dataframes
if dataframes:
    df = pd.concat(dataframes, ignore_index=True)
    
    # Convert match_date to datetime
    df['match_date'] = pd.to_datetime(df['match_date'])

df['match_id'] = df.apply(
    lambda x: f"{x['match_teams']}_{x['match_date'].strftime('%Y%m%d')}", 
    axis=1
)

df

,division_id,division,season_id,season,match_date,match_teams,home_team_id,home_team,away_team_id,away_team,...,home_deep,away_deep,home_ppda,away_ppda,bet365_home_odds,bet365_draw_odds,bet365_away_odds,bet365_u25_odds,bet365_o25_odds,match_id
0,98b8784f6685b7289f583e0ce4b4f6f2,Premier League,3ac445d3cc1d404987efdfcfa42f3bcd,20242025,2024-08-16,Man United - Fulham,f2b82cdbdadf9d3ec47c3a6be66dcfad,Man United,8cd5e94668b139c1f42a89a1e130f3cf,Fulham,...,7.0,3.0,7.379310,10.833333,1.60,4.20,5.25,2.50,1.53,Man United - Fulham_20240816
1,98b8784f6685b7289f583e0ce4b4f6f2,Premier League,3ac445d3cc1d404987efdfcfa42f3bcd,20242025,2024-08-17,Newcastle - Southampton,78e9266876e7649e0a12e3840f5be006,Newcastle,5a884401673693b0bdf379fefb7ec2b2,Southampton,...,4.0,13.0,16.250000,3.789474,1.36,5.25,8.00,3.00,1.40,Newcastle - Southampton_20240817
2,98b8784f6685b7289f583e0ce4b4f6f2,Premier League,3ac445d3cc1d404987efdfcfa42f3bcd,20242025,2024-08-17,Arsenal - Wolves,4fc9baf210346939946d5a49f255588b,Arsenal,31f3ecbc5c48590ccc7dabaedd49a4ff,Wolves,...,14.0,2.0,7.769231,10.818182,1.18,7.50,13.00,2.75,1.44,Arsenal - Wolves_20240817
3,98b8784f6685b7289f583e0ce4b4f6f2,Premier League,3ac445d3cc1d404987efdfcfa42f3bcd,20242025,2024-08-17,Nott'm Forest - Bournemouth,9a8e1e9fad8766fc3d69a0c26d98b928,Nott'm Forest,b436d55f36cfbe8a085c8b75fb7fe98a,Bournemouth,...,10.0,4.0,8.653846,9.954545,2.45,3.50,2.80,2.10,1.73,Nott'm Forest - Bournemouth_20240817
4,98b8784f6685b7289f583e0ce4b4f6f2,Premier League,3ac445d3cc1d404987efdfcfa42f3bcd,20242025,2024-08-17,Ipswich - Liverpool,e4f63bf6d6d2cd121e6c8e59bef68209,Ipswich,afce84ff226407a47c9782a742ba02f7,Liverpool,...,2.0,13.0,18.777778,8.739130,8.50,5.50,1.33,3.00,1.40,Ipswich - Liverpool_20240817
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4468,c4781225ef2d5018f7a9df4b6cb1c1fe,Championship,a290f895d08db6bb30a573aedf784bad,20202021,2021-05-08,Watford - Swansea,6e424e2cb43e11ef176c2a8dcb72f9de,Watford,013d5e8c67e9155030c74930aaf5be33,Swansea,...,NaN,NaN,NaN,NaN,2.00,3.40,3.80,1.85,2.00,Watford - Swansea_20210508
4469,c4781225ef2d5018f7a9df4b6cb1c1fe,Championship,a290f895d08db6bb30a573aedf784bad,20202021,2021-05-08,Middlesbrough - Wycombe,b4cb58c3a936665786359c2c634d6ff4,Middlesbrough,c7e249acd30a69cea63bc51a66e08807,Wycombe,...,NaN,NaN,NaN,NaN,1.75,4.00,4.33,2.10,1.72,Middlesbrough - Wycombe_20210508
4470,c4781225ef2d5018f7a9df4b6cb1c1fe,Championship,a290f895d08db6bb30a573aedf784bad,20202021,2021-05-08,Coventry - Millwall,9ed47047fb3b7f26ada9a35fe83d7b1b,Coventry,ea11a8d754f9bbf49dc01d6d3ae875d9,Millwall,...,NaN,NaN,NaN,NaN,2.37,3.50,2.87,1.80,2.00,Coventry - Millwall_20210508
4471,c4781225ef2d5018f7a9df4b6cb1c1fe,Championship,a290f895d08db6bb30a573aedf784bad,20202021,2021-05-08,Reading - Huddersfield,26fccddf2f94b1685b184267996e10f8,Reading,5668c64f7d3e02ee708188dbe0fdaa4e,Huddersfield,...,NaN,NaN,NaN,NaN,1.95,3.75,3.50,2.00,1.80,Reading - Huddersfield_20210508


In [3]:
def append_df_to_sqlite_table(df_new, db_path, table_name, key_columns=None, batch_size=500):

    # Make a copy to avoid modifying the original dataframe
    df = df_new.copy()
    
    # Connect to the database
    conn = sqlite3.connect(db_path)
    cursor = conn.cursor()
    
    # Check if table exists
    cursor.execute("SELECT name FROM sqlite_master WHERE type='table' AND name=?", (table_name,))
    table_exists = cursor.fetchone() is not None
    
    if table_exists:
        # If key_columns not specified, use all columns
        if key_columns is None:
            key_columns = list(df.columns)
        
        # Get columns from existing table
        cursor.execute(f"PRAGMA table_info({table_name})")
        existing_columns = [row[1] for row in cursor.fetchall()]
        
        # Ensure all key columns exist in both dataframe and table
        key_columns = [col for col in key_columns if col in df.columns and col in existing_columns]
        
        if key_columns:
            # Process in batches to avoid SQL expression tree too large error
            filtered_dfs = []
            
            # Split dataframe into batches
            total_rows = len(df)
            for start_idx in range(0, total_rows, batch_size):
                end_idx = min(start_idx + batch_size, total_rows)
                batch_df = df.iloc[start_idx:end_idx]
                
                # For this batch, extract all key column values
                key_values = {}
                for key in key_columns:
                    key_values[key] = batch_df[key].tolist()
                
                # Build an optimized query - using IN clause for each key column
                where_clauses = []
                params = []
                
                for key in key_columns:
                    if len(key_values[key]) > 0:
                        placeholders = ','.join(['?'] * len(key_values[key]))
                        where_clauses.append(f"{key} IN ({placeholders})")
                        params.extend(key_values[key])
                
                if where_clauses:
                    # Get existing records
                    query = f"SELECT {', '.join(key_columns)} FROM {table_name} WHERE {' OR '.join(where_clauses)}"
                    cursor.execute(query, params)
                    existing_rows = cursor.fetchall()
                    
                    # Create a set of tuples representing existing rows
                    if key_columns:
                        existing_keys = set(existing_rows)
                        
                        # Filter out rows that already exist
                        batch_filtered = batch_df[~batch_df[key_columns].apply(tuple, axis=1).isin(existing_keys)]
                        if not batch_filtered.empty:
                            filtered_dfs.append(batch_filtered)
                else:
                    # No valid key values, include all rows
                    filtered_dfs.append(batch_df)
            
            # Combine all filtered batches
            if filtered_dfs:
                df_filtered = pd.concat(filtered_dfs, ignore_index=True)
                if not df_filtered.empty:
                    df_filtered.to_sql(table_name, conn, if_exists='append', index=False)
                    print(f"Added {len(df_filtered)} new records to table {table_name}")
                else:
                    print("No new records to append.")
            else:
                print("No new records to append.")
        else:
            # If no valid key columns, append all data (may create duplicates)
            df.to_sql(table_name, conn, if_exists='append', index=False)
            print(f"Warning: No valid key columns. Added all {len(df)} records to table {table_name}")
    else:
        # If table doesn't exist, create it with the new data
        df.to_sql(table_name, conn, index=False)
        print(f"Created new table {table_name} with {len(df)} records")
    
    # Close the connection
    conn.close()

append_df_to_sqlite_table(df, 'match_db.db', 'btb_matches', key_columns=['match_id'])

Created new table btb_matches with 4473 records


## Creating XGBoost Model Features

In [4]:
# Separates out the home vs away and allows every team to have an individual row of data. Allows us to feature engineer home effect easier.
home_df = df[["division", "season", "match_date", "match_id",
             "home_team", "home_avg_market_value", "home_goals", "home_shots", "home_shots_on_target", "home_corners", "home_xgoals", "home_deep", "home_ppda", "home_red",
             "away_team", "away_avg_market_value", "away_goals", "away_shots", "away_shots_on_target", "away_corners", "away_xgoals", "away_deep", "away_ppda", "away_red",
              "bet365_home_odds", "bet365_draw_odds", "bet365_away_odds"]].copy()
home_df["home?"] = 1
home_df = home_df.rename(columns={"home_team":"team", "home_avg_market_value":"avg_market_value", "home_goals":"goals",
             "home_shots":"shots", "home_shots_on_target":"shots_on_target", "home_corners":"corners", "home_xgoals":"xG", "home_deep":"deep", "home_ppda":"ppda", "home_red": "red", "bet365_home_odds":"odds",
             "away_team":"opponent_team", "away_avg_market_value":"opponent_avg_market_value", "away_goals":"opponent_goals", 
             "away_shots": "opponent_shots", "away_shots_on_target":"opponent_shots_on_target", "away_corners":"opponent_corners", "away_xgoals": "opponent_xG", 
             "away_deep":"opponent_deep", "away_ppda":"opponent_ppda", "away_red": "opponent_red", "bet365_away_odds":"opponent_odds"})

away_df = df[["division", "season", "match_date", "match_id",
                "away_team", "away_avg_market_value", "away_goals", "away_shots", "away_shots_on_target", "away_corners", "away_xgoals", "away_deep", "away_ppda", "away_red",
                "home_team", "home_avg_market_value", "home_goals", "home_shots", "home_shots_on_target", "home_corners", "home_xgoals", "home_deep", "home_ppda", "home_red",
                 "bet365_home_odds", "bet365_draw_odds", "bet365_away_odds"]].copy()
away_df["home?"] = 0
away_df = away_df.rename(columns={"away_team":"team", "away_avg_market_value":"avg_market_value", "away_goals":"goals",
             "away_shots":"shots", "away_shots_on_target":"shots_on_target", "away_corners":"corners", "away_xgoals":"xG", "away_deep":"deep", "away_ppda":"ppda", "away_red": "red", "bet365_away_odds":"odds",
             "home_team":"opponent_team", "home_avg_market_value":"opponent_avg_market_value", "home_goals":"opponent_goals", 
             "home_shots": "opponent_shots", "home_shots_on_target":"opponent_shots_on_target", "home_corners":"opponent_corners", "home_xgoals": "opponent_xG", 
             "home_deep":"opponent_deep", "home_ppda":"opponent_ppda", "home_red": "opponent_red","bet365_home_odds":"opponent_odds"})

df = pd.concat([away_df, home_df])
df["prem?"] = df["division"].apply(lambda x: 1 if x == "Premier League" else 0)
df["team_season"] = df["team"] + "_" + df["season"].astype(str)
df["opponent_team_season"] = df["opponent_team"] + "_" + df["season"].astype(str)
df.sort_values(["match_date", "division"], inplace=True)

df

,division,season,match_date,match_id,team,avg_market_value,goals,shots,shots_on_target,corners,...,opponent_deep,opponent_ppda,opponent_red,opponent_odds,bet365_draw_odds,odds,home?,prem?,team_season,opponent_team_season
3921,Championship,20202021,2020-09-11,Watford - Middlesbrough_20200911,Middlesbrough,1964000,0,11.0,4,6,...,NaN,NaN,0,2.00,3.30,4.00,0,0,Middlesbrough_20202021,Watford_20202021
3921,Championship,20202021,2020-09-11,Watford - Middlesbrough_20200911,Watford,6487273,1,10.0,2,6,...,NaN,NaN,0,4.00,3.30,2.00,1,0,Watford_20202021,Middlesbrough_20202021
3922,Championship,20202021,2020-09-12,Millwall - Stoke_20200912,Stoke,2382188,0,13.0,4,8,...,NaN,NaN,0,2.50,3.20,2.90,0,0,Stoke_20202021,Millwall_20202021
3923,Championship,20202021,2020-09-12,Bournemouth - Blackburn_20200912,Blackburn,1133200,2,15.0,2,2,...,NaN,NaN,0,1.95,3.50,3.90,0,0,Blackburn_20202021,Bournemouth_20202021
3924,Championship,20202021,2020-09-12,Preston - Swansea_20200912,Swansea,1845385,1,6.0,1,3,...,NaN,NaN,0,2.30,3.25,3.25,0,0,Swansea_20202021,Preston_20202021
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
287,Premier League,20242025,2025-03-16,Arsenal - Chelsea_20250316,Chelsea,25909091,0,8.0,2,4,...,6.0,13.888889,0,1.80,3.60,4.50,0,1,Chelsea_20242025,Arsenal_20242025
288,Premier League,20242025,2025-03-16,Leicester - Man United_20250316,Man United,27917188,3,18.0,5,4,...,6.0,10.538462,0,4.50,3.50,1.83,0,1,Man United_20242025,Leicester_20242025
286,Premier League,20242025,2025-03-16,Fulham - Tottenham_20250316,Fulham,12704545,2,13.0,4,6,...,7.0,10.037037,0,3.60,3.75,2.00,1,1,Fulham_20242025,Tottenham_20242025
287,Premier League,20242025,2025-03-16,Arsenal - Chelsea_20250316,Arsenal,50869565,1,12.0,4,5,...,0.0,11.588235,0,4.50,3.60,1.80,1,1,Arsenal_20242025,Chelsea_20242025


In [5]:
def apply_weighted_avg(col, match_date, match_red, division=None, decay_rate=0.005, time_window=365):
    # Create a mask for non-NaN values
    valid_mask = ~pd.isna(col)
    
    # If all values are NaN, return NaN
    if not valid_mask.any():
        return np.nan
    
    # Filter out NaN values
    valid_col = col[valid_mask].copy()  # Create a copy to avoid modifying original
    valid_dates = match_date[valid_mask]
    valid_red = match_red[valid_mask]
    
    # Get division for valid entries
    valid_division = None
    if division is not None:
        valid_division = division[valid_mask]
    
    # Apply Championship adjustment if division is provided
    if division is not None:
        # Check if this is for an attacking or defensive metric
        is_attacking = 'opponent' not in str(col.name).lower() if hasattr(col, 'name') else False
        is_ppda = 'ppda' in str(col.name).lower() if hasattr(col, 'name') else False
        
        # Create a new numpy array from the Series values
        values_array = valid_col.to_numpy()
        
        # Apply the appropriate adjustment based on metric type
        #for i, div in enumerate(valid_division):
        #    if div == 'Championship':
        #        if is_attacking:
        #            if is_ppda:
        #                values_array[i] *= 1.4  # For PPDA, higher is worse for attack
        #            else:
        #                values_array[i] *= 0.7  # Reduce attacking stats by 40%
        #        else:
        #            if is_ppda:
        #                values_array[i] *= 0.7  # For PPDA, lower is better for defense
        #            else:
        #                values_array[i] *= 1.4  # Increase defensive stats by 40%
        
        # Replace the Series values with the modified array
        valid_col = pd.Series(values_array, index=valid_col.index)
    
    # Get most recent date
    recent_date = max(valid_dates)
    
    # Create a time window mask (only include matches within time_window days)
    time_window_mask = (recent_date - valid_dates).dt.days <= time_window
    
    # If no matches in the time window, return NaN
    if not time_window_mask.any():
        return np.nan
    
    # Apply time window filter
    valid_col = valid_col[time_window_mask]
    valid_dates = valid_dates[time_window_mask]
    valid_red = valid_red[time_window_mask]
    
    if division is not None:
        valid_division = valid_division[time_window_mask]
    
    # Calculate weights for matches within the time window
    match_weight = np.exp(-(recent_date - valid_dates).dt.days * decay_rate)
    
    # Reduce weight for matches with red cards (now using 0.3 instead of 0.5)
    match_weight = np.where(valid_red == 1, match_weight * 0.3, match_weight)
    
    # Get division of the most recent match
    if division is not None:
        # Find the index of the most recent date
        most_recent_idx = valid_dates[valid_dates == recent_date].index
        if len(most_recent_idx) > 0:
            # Get division of the most recent match
            current_division = valid_division.iloc[0] if len(most_recent_idx) == len(valid_division) else valid_division[most_recent_idx[0]]
            # Apply 0.5 weight to matches with different division
            match_weight = np.where(valid_division != current_division, match_weight * 0.5, match_weight)

    # Calculate weighted average
    weighted_avg = sum(match_weight * valid_col) / sum(match_weight) 

    return weighted_avg

In [6]:
# Assuming df is already sorted by match_date
df = df.sort_values(['team', 'match_date'])

def calc_team_metrics_up_to_date(team, current_date, decay_rate=0.0065, time_window=365):
    # Get all matches for the team up to but not including the current date
    team_matches = df[(df['team'] == team) & (df['match_date'] < current_date)]
    
    # If no prior matches, return NaN for all metrics
    if len(team_matches) == 0:
        return pd.Series({
            'rolling_goals_for': np.nan,
            'rolling_goals_against': np.nan,
            'rolling_xg_for': np.nan,
            'rolling_xg_against': np.nan,
            'rolling_shots_for': np.nan,
            'rolling_shots_against': np.nan,
            'rolling_shots_on_target_for': np.nan,
            'rolling_shots_on_target_against': np.nan,
            'rolling_corners_for': np.nan,
            'rolling_corners_against': np.nan,
            'rolling_deep_for': np.nan,
            'rolling_deep_against': np.nan,
            'rolling_ppda_for': np.nan,
            'rolling_ppda_against': np.nan,
            'rolling_odds_for': np.nan,
            'rolling_odds_against': np.nan
        })
    
    # Calculate metrics using the original weighted average function
    metrics = {
        'rolling_goals_for': apply_weighted_avg(
            team_matches['goals'], team_matches['match_date'], team_matches['red'], team_matches['division'],
            decay_rate, time_window
        ),
        'rolling_goals_against': apply_weighted_avg(
            team_matches['opponent_goals'], team_matches['match_date'], team_matches['red'], team_matches['division'],
            decay_rate, time_window
        ),
        'rolling_xg_for': apply_weighted_avg(
            team_matches['xG'], team_matches['match_date'], team_matches['red'], team_matches['division'],
            decay_rate, time_window
        ),
        'rolling_xg_against': apply_weighted_avg(
            team_matches['opponent_xG'], team_matches['match_date'], team_matches['red'], team_matches['division'],
            decay_rate, time_window
        ),
        'rolling_shots_for': apply_weighted_avg(
            team_matches['shots'], team_matches['match_date'], team_matches['red'], team_matches['division'],
            decay_rate, time_window
        ),
        'rolling_shots_against': apply_weighted_avg(
            team_matches['opponent_shots'], team_matches['match_date'], team_matches['red'], team_matches['division'],
            decay_rate, time_window
        ),
        'rolling_shots_on_target_for': apply_weighted_avg(
            team_matches['shots_on_target'], team_matches['match_date'], team_matches['red'], team_matches['division'],
            decay_rate, time_window
        ),
        'rolling_shots_on_target_against': apply_weighted_avg(
            team_matches['opponent_shots_on_target'], team_matches['match_date'], team_matches['red'], team_matches['division'],
            decay_rate, time_window
        ),
        'rolling_corners_for': apply_weighted_avg(
            team_matches['corners'], team_matches['match_date'], team_matches['red'], team_matches['division'],
            decay_rate, time_window
        ),
        'rolling_corners_against': apply_weighted_avg(
            team_matches['opponent_corners'], team_matches['match_date'], team_matches['red'], team_matches['division'],
            decay_rate, time_window
        ),
        'rolling_deep_for': apply_weighted_avg(
            team_matches['deep'], team_matches['match_date'], team_matches['red'], team_matches['division'],
            decay_rate, time_window
        ),
        'rolling_deep_against': apply_weighted_avg(
            team_matches['opponent_deep'], team_matches['match_date'], team_matches['red'], team_matches['division'],
            decay_rate, time_window
        ),
        'rolling_ppda_for': apply_weighted_avg(
            team_matches['ppda'], team_matches['match_date'], team_matches['red'], team_matches['division'],
            decay_rate, time_window
        ),
        'rolling_ppda_against': apply_weighted_avg(
            team_matches['opponent_ppda'], team_matches['match_date'], team_matches['red'], team_matches['division'],
            decay_rate, time_window
        ),
        'rolling_odds_for': apply_weighted_avg(
            team_matches['odds'], team_matches['match_date'], team_matches['red'], team_matches['division'],
            decay_rate, time_window
        ),
        'rolling_odds_against': apply_weighted_avg(
            team_matches['opponent_odds'], team_matches['match_date'], team_matches['red'], team_matches['division'],
            decay_rate, time_window
        ),
    }
    
    return pd.Series(metrics)

def calc_team_metrics_multi_window(team, current_date, windows_and_decays=[(365, 0.0065), (30, 0.001)]):
    # Dictionary to store results for each window
    all_metrics = {}
    
    # Calculate metrics for each time window with its specific decay rate
    for window, decay in windows_and_decays:
        # Use existing function but with specific window and decay rate
        window_metrics = calc_team_metrics_up_to_date(
            team, 
            current_date,
            decay_rate=decay,
            time_window=window
        )
        
        # Add window suffix to column names (only for non-default window)
        window_suffix = f"_{window}d" if window != 365 else ""
        
        # Add these metrics to our results
        for metric, value in window_metrics.items():
            all_metrics[f"{metric}{window_suffix}"] = value
    
    return pd.Series(all_metrics)

# Apply the function to each row for team stats
team_results = []
opponent_results = []

# Define window sizes and decay rates
windows_and_decays = [(365, 0.0065), (30, 0.001)]

for idx, row in df.iterrows():
    # Calculate team metrics with multiple windows
    team_metrics = calc_team_metrics_multi_window(
        row['team'], 
        row['match_date'], 
        windows_and_decays=windows_and_decays
    )
    
    # Calculate opponent metrics with multiple windows
    opponent_metrics_raw = calc_team_metrics_multi_window(
        row['opponent_team'], 
        row['match_date'],
        windows_and_decays=windows_and_decays
    )
    
    # Rename opponent metrics with a clearer naming convention
    opponent_metrics = {}
    for metric, value in opponent_metrics_raw.items():
        opponent_metrics[f"opponent_{metric}"] = value
    
    team_results.append(team_metrics)
    opponent_results.append(pd.Series(opponent_metrics))

# Create DataFrames from the results
team_metrics_df = pd.DataFrame(team_results, index=df.index)
opponent_metrics_df = pd.DataFrame(opponent_results, index=df.index)

# Check for duplicated columns before concatenation
print("Team metrics columns:", team_metrics_df.columns.tolist())
print("Opponent metrics columns:", opponent_metrics_df.columns.tolist())

# Combine everything into the final DataFrame
final_df = pd.concat([df, team_metrics_df, opponent_metrics_df], axis=1)


final_df

Team metrics columns: ['rolling_goals_for', 'rolling_goals_against', 'rolling_xg_for', 'rolling_xg_against', 'rolling_shots_for', 'rolling_shots_against', 'rolling_shots_on_target_for', 'rolling_shots_on_target_against', 'rolling_corners_for', 'rolling_corners_against', 'rolling_deep_for', 'rolling_deep_against', 'rolling_ppda_for', 'rolling_ppda_against', 'rolling_odds_for', 'rolling_odds_against', 'rolling_goals_for_30d', 'rolling_goals_against_30d', 'rolling_xg_for_30d', 'rolling_xg_against_30d', 'rolling_shots_for_30d', 'rolling_shots_against_30d', 'rolling_shots_on_target_for_30d', 'rolling_shots_on_target_against_30d', 'rolling_corners_for_30d', 'rolling_corners_against_30d', 'rolling_deep_for_30d', 'rolling_deep_against_30d', 'rolling_ppda_for_30d', 'rolling_ppda_against_30d', 'rolling_odds_for_30d', 'rolling_odds_against_30d']
Opponent metrics columns: ['opponent_rolling_goals_for', 'opponent_rolling_goals_against', 'opponent_rolling_xg_for', 'opponent_rolling_xg_against', 'opp

,division,season,match_date,match_id,team,avg_market_value,goals,shots,shots_on_target,corners,...,opponent_rolling_shots_on_target_for_30d,opponent_rolling_shots_on_target_against_30d,opponent_rolling_corners_for_30d,opponent_rolling_corners_against_30d,opponent_rolling_deep_for_30d,opponent_rolling_deep_against_30d,opponent_rolling_ppda_for_30d,opponent_rolling_ppda_against_30d,opponent_rolling_odds_for_30d,opponent_rolling_odds_against_30d
1431,Premier League,20202021,2020-09-12,Fulham - Arsenal_20200912,Arsenal,19978333,3,5.0,2,2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1438,Premier League,20202021,2020-09-19,Arsenal - West Ham_20200919,Arsenal,20354688,2,NaN,14,13,...,15.000000,3.000000,7.000000,8.000000,5.000000,10.000000,6.410000,24.180000,2.150000,3.400000
1456,Premier League,20202021,2020-09-28,Liverpool - Arsenal_20200928,Arsenal,20354688,1,21.0,8,7,...,5.303423,8.786309,4.839038,8.303423,17.606846,1.767808,10.288289,15.903367,1.493617,8.013969
1466,Premier League,20202021,2020-10-04,Arsenal - Sheffield United_20201004,Arsenal,20217188,2,NaN,6,9,...,6.505963,3.627549,8.309350,11.304476,5.187764,11.562909,14.738665,10.302835,2.833680,2.623453
1468,Premier League,20202021,2020-10-17,Man City - Arsenal_20201017,Arsenal,20385484,0,13.0,5,6,...,5.011988,14.018054,6.339323,7.674695,8.002018,3.670671,8.110910,21.160884,1.379721,7.835323
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4415,Championship,20202021,2021-04-17,Swansea - Wycombe_20210417,Wycombe,277941,2,15.0,5,5,...,2.340780,3.320107,5.143635,4.335824,NaN,NaN,NaN,NaN,2.630254,2.989631
4430,Championship,20202021,2021-04-21,Wycombe - Bristol City_20210421,Wycombe,277941,2,22.0,6,5,...,1.607782,3.697505,4.168051,3.618023,NaN,NaN,NaN,NaN,3.258108,2.343212
4439,Championship,20202021,2021-04-24,Cardiff - Wycombe_20210424,Wycombe,277941,1,15.0,2,6,...,4.001784,4.406553,4.790115,4.610477,NaN,NaN,NaN,NaN,3.212021,2.807037
4459,Championship,20202021,2021-05-01,Wycombe - Bournemouth_20210501,Wycombe,277941,1,13.0,4,4,...,4.570036,2.863177,4.137542,3.996018,NaN,NaN,NaN,NaN,2.284378,3.726279


In [7]:
append_df_to_sqlite_table(final_df, 'match_db.db', 'xgb_match_features_0065_001', key_columns=['match_id'])

final_df

Created new table xgb_match_features_0065_001 with 8946 records


,division,season,match_date,match_id,team,avg_market_value,goals,shots,shots_on_target,corners,...,opponent_rolling_shots_on_target_for_30d,opponent_rolling_shots_on_target_against_30d,opponent_rolling_corners_for_30d,opponent_rolling_corners_against_30d,opponent_rolling_deep_for_30d,opponent_rolling_deep_against_30d,opponent_rolling_ppda_for_30d,opponent_rolling_ppda_against_30d,opponent_rolling_odds_for_30d,opponent_rolling_odds_against_30d
1431,Premier League,20202021,2020-09-12,Fulham - Arsenal_20200912,Arsenal,19978333,3,5.0,2,2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1438,Premier League,20202021,2020-09-19,Arsenal - West Ham_20200919,Arsenal,20354688,2,NaN,14,13,...,15.000000,3.000000,7.000000,8.000000,5.000000,10.000000,6.410000,24.180000,2.150000,3.400000
1456,Premier League,20202021,2020-09-28,Liverpool - Arsenal_20200928,Arsenal,20354688,1,21.0,8,7,...,5.303423,8.786309,4.839038,8.303423,17.606846,1.767808,10.288289,15.903367,1.493617,8.013969
1466,Premier League,20202021,2020-10-04,Arsenal - Sheffield United_20201004,Arsenal,20217188,2,NaN,6,9,...,6.505963,3.627549,8.309350,11.304476,5.187764,11.562909,14.738665,10.302835,2.833680,2.623453
1468,Premier League,20202021,2020-10-17,Man City - Arsenal_20201017,Arsenal,20385484,0,13.0,5,6,...,5.011988,14.018054,6.339323,7.674695,8.002018,3.670671,8.110910,21.160884,1.379721,7.835323
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4415,Championship,20202021,2021-04-17,Swansea - Wycombe_20210417,Wycombe,277941,2,15.0,5,5,...,2.340780,3.320107,5.143635,4.335824,NaN,NaN,NaN,NaN,2.630254,2.989631
4430,Championship,20202021,2021-04-21,Wycombe - Bristol City_20210421,Wycombe,277941,2,22.0,6,5,...,1.607782,3.697505,4.168051,3.618023,NaN,NaN,NaN,NaN,3.258108,2.343212
4439,Championship,20202021,2021-04-24,Cardiff - Wycombe_20210424,Wycombe,277941,1,15.0,2,6,...,4.001784,4.406553,4.790115,4.610477,NaN,NaN,NaN,NaN,3.212021,2.807037
4459,Championship,20202021,2021-05-01,Wycombe - Bournemouth_20210501,Wycombe,277941,1,13.0,4,4,...,4.570036,2.863177,4.137542,3.996018,NaN,NaN,NaN,NaN,2.284378,3.726279


In [ ]:
df = pd.read_csv("epl_odds.csv")
df["match_id"] = df["HomeTeam"] + ' - ' + df["AwayTeam"] + '_' + pd.to_datetime(df["Date"], format="%d/%m/%Y").dt.strftime("%Y%m%d")

,Div,Date,Time,HomeTeam,AwayTeam,FTHG,FTAG,FTR,HTHG,HTAG,...,B365CAHA,PCAHH,PCAHA,MaxCAHH,MaxCAHA,AvgCAHH,AvgCAHA,BFECAHH,BFECAHA,match_id
0,E0,16/08/2024,20:00,Man United,Fulham,1,0,H,0,0,...,2.07,1.83,2.11,1.88,2.11,1.82,2.05,1.90,2.08,Man United - Fulham_20240816
1,E0,17/08/2024,12:30,Ipswich,Liverpool,0,2,A,0,0,...,1.88,2.04,1.90,2.20,2.00,1.99,1.88,2.04,1.93,Ipswich - Liverpool_20240817
2,E0,17/08/2024,15:00,Arsenal,Wolves,2,0,H,1,0,...,1.91,2.00,1.90,2.05,1.93,1.99,1.87,2.02,1.96,Arsenal - Wolves_20240817
3,E0,17/08/2024,15:00,Everton,Brighton,0,3,A,0,1,...,2.06,1.86,2.07,1.92,2.10,1.83,2.04,1.88,2.11,Everton - Brighton_20240817
4,E0,17/08/2024,15:00,Newcastle,Southampton,1,0,H,1,0,...,2.06,1.88,2.06,1.89,2.10,1.82,2.05,1.89,2.10,Newcastle - Southampton_20240817
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
284,E0,15/03/2025,15:00,Southampton,Wolves,1,2,A,0,1,...,2.07,1.85,2.08,1.89,2.08,1.83,2.04,1.89,2.10,Southampton - Wolves_20250315
285,E0,15/03/2025,17:30,Bournemouth,Brentford,1,2,A,1,1,...,1.97,1.96,1.96,1.99,1.97,1.94,1.92,2.00,1.98,Bournemouth - Brentford_20250315
286,E0,16/03/2025,13:30,Arsenal,Chelsea,1,0,H,1,0,...,1.95,1.96,1.96,1.97,1.97,1.94,1.92,1.96,2.02,Arsenal - Chelsea_20250316
287,E0,16/03/2025,13:30,Fulham,Tottenham,2,0,H,0,0,...,2.06,1.85,2.08,1.86,2.09,1.84,2.02,1.90,2.10,Fulham - Tottenham_20250316


In [11]:
conn = sqlite3.connect('match_db.db')
conn.execute("DROP TABLE IF EXISTS epl_odds")
df.to_sql('epl_odds', conn, if_exists='replace', index=False)
conn.close()